# Task 4: The Challenge

## Prepare Dataset

In [1]:
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
from tqdm import tqdm

from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, auc, confusion_matrix, make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:

from operator import itemgetter

import itertools

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from numpy.linalg import norm
from scipy.optimize import differential_evolution
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

from statsmodels.graphics.gofplots import qqplot
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score
from sklearn.metrics import recall_score, precision_score, confusion_matrix

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, GroupKFold, ParameterGrid
sns.set()

In [3]:
# load datasets

df_train = pd.read_csv("task_3_training_e8da4715deef7d56_f8b7378_generic.csv", header=0)
df_test = pd.read_csv("task_4_test_dd4bd32b08b776e6_daf99ad_generic.csv", header=0)

# keep high level features (GEMS) as separate array for training

hl_train = df_train[df_train.columns[175:]]

# drop columns 175-201, since they contain the high-level features not available in the test set

df_train = df_train.drop(df_train.columns[175:], axis=1)

# drop valence and arousal as specified in the task description as well as pianist, segment and snippet column

df_train = df_train.drop(["arousal", "valence", "pianist_id", "segment_id", "snippet_id"], axis=1)
df_test_ids = df_test[["pianist_id", "segment_id", "snippet_id"]]
df_test = df_test.drop(["pianist_id", "segment_id", "snippet_id"], axis=1)

In [4]:
df_test_ids.shape

(2085, 3)

In [5]:
df_test.shape, df_train.shape

((2085, 169), (2282, 170))

In [6]:
# separate data and labels from training set

X_train = df_train[df_train.columns[:-1]]
y_train = df_train[df_train.columns[-1]]

# this adjustment is because of the XGBoost, it needs value for multiclass in 0... num of class - 1
y_train = y_train -1

## Predicting high-level features

In [ ]:
# normalize data

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(df_test)

# use non-optimized xgb to predict high level features
# multioutputregressor trains one regressor for each high level feature

xgb_multi = MultiOutputRegressor(xgb.XGBRegressor(objective="reg:squarederror", random_state=42), n_jobs=-1)
xgb_multi.fit(X_train, hl_train)
hl_pred = xgb_multi.predict(X_test)

In [ ]:
hl_pred

In [ ]:
# merge high level features together with test set
columns = ["gems_wonder","gems_transcendence","gems_tenderness","gems_nostalgia","gems_peacefulness","gems_power","gems_joyful_activation","gems_tension","gems_sadness","gemmes_movement","gemmes_force","gemmes_interior","gemmes_wandering","gemmes_flow","gems_wonder_binary","gems_transcendence_binary","gems_tenderness_binary","gems_nostalgia_binary","gems_peacefulness_binary","gems_power_binary","gems_joyful_activation_binary","gems_tension_binary","gems_sadness_binary","gemmes_movement_binary","gemmes_force_binary","gemmes_interior_binary","gemmes_wandering_binary","gemmes_flow_binary"]
df_test = pd.concat([df_test, pd.DataFrame(hl_pred, columns=columns)], axis=1)
df_test.to_csv("df_test_restored_hl.csv", index=False)

# df_test exploration

In [ ]:
df_test

In [ ]:
fig = plt.gcf()
fig.set_size_inches(18.5, 15.5)
sub_df = df_test.astype('float64')
l = sub_df.columns.shape[0]
sub_df.hist(figsize=(40,70),layout=( 57 ,4),color='indigo');

In [ ]:
#calculating pearson correlations and taking only pair where significant noticed
corr = df_test.astype('float64'
                   ).corr(method='pearson')
redundant = []
for column in corr.columns:
    n_larg = corr[column].nlargest(3)
    n_smal = corr[column].nsmallest(3)
    if abs(n_larg[1])>0.75:
        print(f"{'='*60}\nFeature {column} is most correlated with\n")
        print(n_larg)
        if abs(n_smal[0])>0.75:
            print(n_smal)
    elif abs(n_smal[1])>0.75:
        print(f"{'='*60}\nFeature {column} is most correlated with\n")
        print(n_smal)
    else:
        redundant.append(column)
print(f"{'='*60}\nFollowing fetures have no significant correlations:\n{redundant}")

In [ ]:
def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

In [ ]:
get_top_abs_correlations(df_test, n=50)

## Predict quadrants

# There shoul be saciling but i discovered it lately. It works even without, but classifiers,
# than classiers are confused. But they work, not best.
# Never mind, do not care about it, do only what is mandatory for us. They do not check a code.

In [ ]:
# reload df_train to restore high level features removed earlier and drop non usable columns and get df_test from saved file (so the notebook doesn't need to be run every time)

df_train = pd.read_csv("task_3_training_e8da4715deef7d56_f8b7378_generic.csv", header=0)
df_test = pd.read_csv("df_test_restored_hl.csv", header=0)

#There shoul be saciling but i discovered it lately. It works even without, but classifiers,
# than classiers are confused. But they work, not best.
# Never mind, do not care about it, do only what is mandatory for us. They do not check a code.
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# save segment_id column for GroupShuffleSplit and drop pianist, segment and snippet column
group_idx = df_train["segment_id"]

df_train = df_train.drop(["arousal", "valence", "pianist_id", "segment_id", "snippet_id"], axis=1)

In [ ]:
# The last 14 columns (GEMS) are binary. So far we predicted values in range [0,1]. We round the values to match the binary type

df_test[df_test.columns[-14:]] = df_test[df_test.columns[-14:]].round(decimals=0).astype(int)

In [ ]:
# prepare data and label

X = df_train.drop(["quadrant"], axis=1)
y = df_train["quadrant"]

# this adjustment is because of the XGBoost, it needs value for multiclass in 0... num of class - 1
y = y -1

In [ ]:
# custom scoring function for GridSearchCV

# Labels and corresponding mood
# 1: happy
# 2: angry
# 3: sad
# 4: relaxed

def filmotion_scoring(y, y_pred):
    balance = 0
    revenue_matrix = np.array([[5,-5,-5,2],[-5,10,2,-5],[-5,2,10,-5],[2,-5,-2,5]])
    for true, pred in zip(y, y_pred):
        balance += revenue_matrix[int(true)-1,int(pred)-1]
    return balance/len(y)

In [ ]:
mask = np.logical_or(np.logical_or(group_idx==20, group_idx==21), group_idx==23)

# define validation set
X_valid = X.loc[mask]
y_valid = y.loc[mask]
group_idx_valid = group_idx.loc[mask]

# define training set
mask = np.invert(mask)
X_train = X.loc[mask]
y_train = y.loc[mask]
group_idx_train = group_idx.loc[mask]

In [ ]:
# we will use GroupKFold which seperates our data into k folds in which the groups differ, in a way that in 
# the first fold only group 1,2 in the second fold only group 3,4 ... are listed
gkf = GroupKFold(n_splits=6)

In [ ]:
# make custom scorer for grid search using previously defined scoring function
filmotion = make_scorer(filmotion_scoring)

In [ ]:
def sensitivity_matric(yTest, yPred, write=True):
    c_matrix = confusion_matrix(yTest, yPred)
    tmp = 0 
    for i in range(4):
        tmp += c_matrix[i,i] / sum(c_matrix[i,:])
        if write:
            print(f'sensitivity of classier  {i}. is: {c_matrix[i,i] / sum(c_matrix[i,:])}' )
    return tmp 

In [ ]:
# Getting the best parameter settings using GridSearch

#########
## KNN ##
#########

knn_params = {"n_neighbors" : [3, 5, 25, 50], "algorithm": ["auto"], "weights": ["uniform"]}

knn_model = GridSearchCV(KNeighborsClassifier(), param_grid=knn_params, cv=gkf, scoring=filmotion)
knn_model.fit(X_train, y_train, groups=group_idx_train)
knn_param = knn_model.best_params_
knn_model = knn_model.best_estimator_
print("KNN has sensitivity")
knn_prob = knn_model.predict(X_valid)
sensitivity_matric(y_valid, knn_prob)
print()


#########
## XGB ##
#########

xgb_params = {'min_child_weight': [4], 'gamma': [0], 'max_depth': [4],
             'n_estimators': [800]}

xgb_model = GridSearchCV(estimator=XGBClassifier(use_label_encoder=False, eval_metric='merror'), param_grid=xgb_params,
                         cv=gkf)
xgb_model.fit(X_train, y_train, groups=group_idx_train)
xgb_param = xgb_model.best_params_
xgb_model = xgb_model.best_estimator_

print("XGBoost has sensitivity")
xgb_prob = xgb_model.predict(X_valid)
sensitivity_matric(y_valid, xgb_prob)
print()
########
## RF ##
########

rf_params = { 
    'n_estimators': [200],
    'max_features': ['auto'],
    'max_depth' : [4,],
    'criterion' :['gini']
}

rf_model = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params, cv=gkf, scoring=filmotion)
rf_model.fit(X_train, y_train, groups=group_idx_train)
rf_param = rf_model.best_params_
rf_model = rf_model.best_estimator_
print("RandomForest has sensitivity")
rf_prob = rf_model.predict(X_valid)
sensitivity_matric(y_valid, rf_prob)
print()

#########################
## Logistic Regression ##
#########################

log_reg_param = {"penalty": ['l2'], 'C': [1], "dual":[True]}
reg_model = GridSearchCV(LogisticRegression(solver='liblinear', max_iter=8000), param_grid=log_reg_param, cv=gkf, scoring=filmotion)
reg_model.fit(X_train, y_train, groups=group_idx_train)
reg_param = reg_model.best_params_
reg_model = reg_model.best_estimator_
print("LogisticRegression has sensitivity")
reg_prob = reg_model.predict(X_valid)
sensitivity_matric(y_valid, reg_prob)
print()

In [ ]:
print("LogisiticRegression has param:", reg_param)
print("KNerest has param:", knn_param)
print("DecisionTreeClassifier has param:", rf_param)
print("XGBClassifier has param:", xgb_param)


In [ ]:
reg_param["solver"] = 'liblinear'
xgb_param["use_label_encoder"] = False
xgb_param["objective"] = "multi:softmax"
xgb_param["num_class"] = "4"

In [ ]:


importance_dataset = pd.DataFrame({'Features': X.columns, 'Feature importance': rf_model.feature_importances_})
importance_dataset = importance_dataset.sort_values(by=['Feature importance'],ascending=False).reset_index(drop=True)
importance_dataset = importance_dataset[:20]
plt.title('RandomForestClassifier features importance', size=20)
ax = sns.barplot(x="Features", y="Feature importance", data=importance_dataset)

ax.set_xlabel("Features",fontsize=10)
ax.tick_params(axis='x', labelsize=10, rotation=90)
ax.set_ylabel("Feature importance",fontsize=20)
ax.tick_params(axis='y', labelsize=10, rotation=0)



In [ ]:


importance_dataset = pd.DataFrame({'Features': X.columns, 'Feature importance': xgb_model.feature_importances_})
importance_dataset = importance_dataset.sort_values(by=['Feature importance'],ascending=False).reset_index(drop=True)
importance_dataset = importance_dataset[:20]
plt.title('XGBClassifier feature importance', size=20)
ax = sns.barplot(x="Features", y="Feature importance", data=importance_dataset)

ax.set_xlabel("Features",fontsize=10)
ax.tick_params(axis='x', labelsize=10, rotation=90)
ax.set_ylabel("Feature importance",fontsize=20)
ax.tick_params(axis='y', labelsize=10, rotation=0)



In [ ]:


importance = reg_model.coef_[0]
columns = X.columns

im_df = pd.DataFrame({"Feature importance": importance, "Features": columns })

print((im_df["Feature importance"].abs() > 0.0003).value_counts())
im_df = im_df[im_df["Feature importance"].abs() > 0.0003]
plt.title('LogisticRegressor feature importance', size=20)
ax = sns.barplot(x="Features", y="Feature importance", data=im_df)

ax.set_xlabel("Features",fontsize=10)
ax.tick_params(axis='x', labelsize=10, rotation=90)
ax.set_ylabel("Feature importance",fontsize=20)
ax.tick_params(axis='y', labelsize=10, rotation=0)





The ensemble works similar to gradient descend. It tries to find the global optimum for coefficients. So we want to find the best weight for each model, that gives us the best result by summing the result of multiplying weights and model prediction.


In [ ]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
    # make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = np.array(yhats)
    # weighted sum across ensemble members
    summed = np.tensordot(yhats, weights, axes=((0),(0)))
    return summed.round()

# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testy):
    # make prediction
    yhat = ensemble_predictions(members, weights, testX)
    # calculate AUC
   
    
    return sensitivity_matric(testy, yhat, write=False)


# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result

# loss function for optimization process, designed to be minimized
def loss_function(weights, members, testX, testy):
    # normalize weights
    normalized = normalize(weights)
    # calculate error rate
    return 1.0 - evaluate_ensemble(members, normalized, testX, testy)


In [ ]:
testX = X_valid
testy = y_valid

members = [reg_model, knn_model, rf_model, xgb_model]

for i in range(len(members)):
    test_prob = members[i].predict(testX)
   
    print('Model %d: %.3f' % (i+1, accuracy_score(testy, test_prob)))
    
# evaluate averaging ensemble (equal weights)
weights = [1.0/len(members) for _ in range(len(members))]
score = evaluate_ensemble(members, weights, testX, testy)


print('Equal Weights Score: %.3f' % score)
# define bounds on each weight
bound_w = [(0.0, 1.0)  for _ in range(len(members))]
# arguments to the loss function
search_arg = (members, testX, testy)
# global optimization of ensemble weights

result = differential_evolution(loss_function, bound_w, search_arg, maxiter=300, tol=1e-7)
# get the chosen weights
weights = normalize(result['x'])
print('Optimized Weights: %s' % weights)
# evaluate chosen weights
score = evaluate_ensemble(members, weights, testX, testy)
print('Optimized Weights AUC: %.3f' % score)


In [ ]:
# get the chosen weights
weights = normalize(result['x'])
print('Optimized Weights: %s' % weights)
# evaluate chosen weights



In [ ]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
#     axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
#                          train_scores_mean + train_scores_std, alpha=0.1,
#                          color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
#     axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
#                  label="Training score")
  
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
plot_learning_curve(LogisticRegression(**reg_param), "LogisticRegression model", X_train, y_train)

In [ ]:
plot_learning_curve(RandomForestClassifier(**rf_param), "Random Forest model", X_train, y_train)

In [ ]:
plot_learning_curve(XGBClassifier(**xgb_param), "XGBoost model", X_train, y_train)

In [ ]:


plot_learning_curve(KNeighborsClassifier(**knn_param), "KNN model", X_train, y_train)



In [ ]:
def plot_confusion_matrix(testY, predY, key="name"):
    fig, ax = plt.subplots(figsize=(8, 8))
    cmp = ConfusionMatrixDisplay(
        confusion_matrix(testY, predY),
        display_labels=["class_1", "class_2", "class_3", "class_4"]
        )

    plt.grid(False)
    plt.title(key, size=20)
    cmp.plot(ax=ax)
    plt.show()

In [ ]:
plot_confusion_matrix(y_valid, reg_model.predict(X_valid), key="LogisticRegression")

In [ ]:
plot_confusion_matrix(y_valid, knn_model.predict(X_valid), key="KNN")

In [ ]:
plot_confusion_matrix(y_valid, rf_model.predict(X_valid), key="RandomForest")

In [ ]:
plot_confusion_matrix(y_valid, xgb_model.predict(X_valid), key="XGBoost")

In [ ]:
plot_confusion_matrix(y_valid, ensemble_predictions(members, weights, X_valid), key="Ensemble model")

# Final training

In [ ]:
X_valid.shape, X_train.shape

In [ ]:
y_valid.shape, y_train.shape

In [ ]:
X_train_concat = pd.concat([X_valid, X_train], axis = 0)
y_train_concat = pd.concat([y_valid, y_train], axis = 0)

In [ ]:
X_train_concat.shape

In [ ]:
X_train

In [ ]:
reg_model = LogisticRegression(**reg_param)
rf_model = RandomForestClassifier(**rf_param)
xgb_model = XGBClassifier(**xgb_param)
knn_model = KNeighborsClassifier(**knn_param)

reg_model.fit(X_train_concat, y_train_concat)
rf_model.fit(X_train_concat, y_train_concat)
xgb_model.fit(X_train_concat, y_train_concat)
knn_model.fit(X_train_concat, y_train_concat)

In [ ]:
# Final prediction
members = [reg_model, knn_model, rf_model, xgb_model]

y_test = ensemble_predictions(members, weights, df_test)

In [ ]:
# we did this because of xgboost, but now we need adjust it again 
y_test = y_test + 1

In [ ]:
y_test = pd.Series(y_test)
y_test = pd.concat([df_test_ids ,y_test], axis=1)


In [ ]:
y_test.columns = ["pianist_id", "segment_id", "snippet_id", "quadrant"]
y_test